In [5]:
from downloaddata import DlStock
import pandas as pd
import os
from tqdm import tqdm
import shutil
import time
import random

## 下载浦发银行数据
一个简单的案例说明

In [2]:
dlstock_object = DlStock(stock_id='600000', start_date='1999-11-10', end_date='2021-10-25')
data1 = dlstock_object.run()
data1.head()

downloading...: 100%|██████████| 89/89 [00:03<00:00, 24.55it/s]


,日期,开盘价,最高价,最低价,收盘价,涨跌额,涨跌幅(%),成交量(手),成交金额(万元),振幅(%),换手率(%)
0,1999-11-10,29.50,29.80,27.00,27.75,17.75,177.50,1740851,485910,28.00,54.40
1,1999-11-11,27.58,28.38,27.53,27.71,-0.04,-0.14,294035,82158,3.06,9.19
2,1999-11-12,27.86,28.30,27.77,28.05,0.34,1.23,150080,42159,1.91,4.69
3,1999-11-15,28.20,28.25,27.70,27.75,-0.30,-1.07,119211,33295,1.96,3.73
4,1999-11-16,27.88,27.97,26.48,26.55,-1.20,-4.32,232231,62891,5.37,7.26


# 如何批量下载上海证券交易所的A股的所有数据
## 方法1. 使用for循环下载数据

In [3]:
dir_name = "股票下载文件夹"
if os.path.exists(dir_name):
    shutil.rmtree(path=dir_name)
    os.mkdir(path=dir_name)
else:
    os.mkdir(path=dir_name)


In [4]:
sz_A_stock_data = pd.read_csv("上海证券交易所股票名单/主板A股.xls",
                              sep='\t',
                              encoding='gb2312')
sz_A_stock_data.columns = [i.strip() for i in sz_A_stock_data.columns.tolist()]
sz_A_stock_data.head()

,公司代码,公司简称,代码,简称,上市日期,Unnamed: 5
0,600000,浦发银行,600000,浦发银行,1999-11-10,
1,600004,白云机场,600004,白云机场,2003-04-28,
2,600006,东风汽车,600006,东风汽车,1999-07-27,
3,600007,中国国贸,600007,中国国贸,1999-03-12,
4,600008,首创环保,600008,首创环保,2000-04-27,


In [44]:
for index in tqdm(range(0, 10)):  #tqdm(range(0, sz_A_stock_data.shape[0])):
    temp_data = sz_A_stock_data.iloc[index]
    temp_dlstock = DlStock(stock_id=temp_data['公司代码'],
                           start_date=temp_data['上市日期'].strip(),
                           end_date='2021-10-25')
    temp_result = temp_dlstock.run()
    temp_result.to_csv(f"{dir_name}/{temp_data['公司简称'].strip()}_{temp_data['公司代码']}.csv", index=False)

  3%|▎         | 43/1645 [01:56<1:12:27,  2.71s/it]


KeyboardInterrupt: 

## 方法2.使用并行下载数据



In [6]:
from joblib import Parallel, delayed

dir_name = "股票下载文件夹"
if os.path.exists(dir_name):
    shutil.rmtree(path=dir_name)
    os.mkdir(path=dir_name)
else:
    os.mkdir(path=dir_name)

sz_A_stock_data = pd.read_csv("上海证券交易所股票名单/主板A股.xls",
                              sep='\t',
                              encoding='gb2312')
sz_A_stock_data.columns = [i.strip() for i in sz_A_stock_data.columns.tolist()]
sz_A_stock_data.head()


def dl_by_parallel(index):
    try:
        time.sleep(random.randint(0, 10))
        temp_data = sz_A_stock_data.iloc[index]
        temp_dlstock = DlStock(stock_id=temp_data['公司代码'],
                               start_date=temp_data['上市日期'].strip(),
                               end_date='2021-10-25')
        temp_result = temp_dlstock.run()
        temp_result.to_csv(f"{dir_name}/{temp_data['公司简称'].strip()}_{temp_data['公司代码']}.csv", index=False)

    except Exception as e:
        index = 0
        pass
    finally:
        return index


check_index_list = Parallel(n_jobs=10)(delayed(dl_by_parallel)(index)
                                       for index in tqdm(range(0, sz_A_stock_data.shape[0])))
print('done~')

100%|██████████| 1645/1645 [31:35<00:00,  1.15s/it] 


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGKILL(-9)}